# Processing data after running the Genetic Algorithm
First, we will look at what values the genetic algorithm gave us.   
Then, we will backcalculate the fold changes for each set of model parameters

In [ ]:
import tellurium as te
import pandas as pd
import numpy as np
import sys
import matplotlib.pyplot as plt

# pd.set_option("display.max_rows", None, "display.max_columns", None)
# pd.reset_option('^display.', silent=True)

# Viewing the data using pandas

In [ ]:
# open the results file
f = open("paramData.list", "r")
lines = f.read()
# parse the data into an array 
list_of_lines = lines.splitlines()
f.close()
# reshape into numpy
arr = np.array(list_of_lines)
arr = arr.astype('float64')
arr = np.reshape(arr, (100, 5))

g = open("scores.list", "r")
gines = g.read()
gist_of_gines = gines.splitlines()
g.close()
garr = np.array(gist_of_gines)
garr = garr.astype('float64')
garr = np.reshape(garr, (100, 1))
gf = pd.DataFrame(garr, columns = ['chiSq'])

data = pd.concat((df, gf),axis=1)
data

In [ ]:
# to view data from least to greatest chi square values
data.sort_values(by='chiSq')

# Backcalculating the predicted fold changes for each set of model parameters

In [6]:
import backCheck
from core import evaluate
from core import models

In [ ]:
backCheck.getFoldChangeValues(paramData="data/paramData.list")

In [7]:
# the original, expected values to compare with the values in 'allData.list'
evaluate.runExperiment(models.groundTruth_mod_e)

array([ 0.69887834,  0.69887834,  0.69887834,  0.69887834, -0.34168351,
        0.0736096 ,  0.0736096 ,  0.0736096 , -0.07181795, -0.37015035,
        0.01547189,  0.01547189, -0.01774854, -0.0914761 , -0.39242256,
        0.0038236 , -0.00471971, -0.02432542, -0.10435341, -0.49949161,
        0.69887834,  0.0736096 ,  0.01547189,  0.0038236 ,  0.00101678])